In [1]:
#!pip install -U aeon
#!pip install aeon[all_extras]
import warnings
import pandas as pd
from matplotlib import pyplot as plt
from aeon.visualisation import plot_series
from all_functions import *
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb 
from sklearn.model_selection import GridSearchCV
from aeon.transformations.detrend import STLTransformer
import ast
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error, make_scorer
import os
from sklearn.linear_model import Ridge, RidgeCV
from datetime import datetime
from distutils.util import strtobool

warnings.filterwarnings("ignore")
%matplotlib inline
def convert_to_list(series_str):
    return eval(series_str)

def get_preds_hybrid(path, test_date, start_index):
    df = pd.read_csv(path, sep=";")
    results = {}
    filtered_df = df[df['test_range'] == test_date]
    columns_p1_to_p12 = filtered_df.loc[:, 'P1':'P12']
    values_list = columns_p1_to_p12.values.flatten().tolist()     
    results = pd.Series(values_list, index=start_index)
    return results

def generate_index(start_date, end_date):
    end_date_dt = pd.to_datetime(end_date)
    
    start_date_dt = pd.to_datetime(start_date)
    
    index = pd.period_range(start=start_date_dt, end=end_date_dt, freq='M')

    return index

/home/user/miniconda3/envs/lucas/lib/python3.11/site-packages/rapids_dask_dependency/dask_loader.py:36: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  mod = importlib.import_module(spec.name)


In [12]:
modelos = [
    "ETS",
    "arima", 
    # "catboost", "ridge",
    # "SWT_ridge", "CWT_ridge", "DWT_ridge", "FIRTS_ridge", "STFT_ridge", "RP_ridge", "MTF_ridge", "WPT_ridge", "GASF_ridge", "GADF_ridge"
    ]
derivados = [
            "gasolinac", "etanolhidratado", "oleodiesel", "glp", 
            #  "querosenedeaviacao"
            ]
transformations = ["normal", "deseasonal"]
dates = ["2019-03_2020-02","2020-03_2021-02","2021-03_2022-02","2022-03_2023-02","2023-03_2024-02"]


for modelo in modelos:
    print(modelo)
    for derivado in derivados:
        for transformation in transformations:
            path_folders = f'./paper_roma/{modelo}/{derivado}/{transformation}/'
            all_mape = []
            all_pocid = []
            all_pbe = []
            all_rmse = []
            if os.path.exists(path_folders):
                for filename in os.listdir(path_folders):
                    all_preds = []
                    all_test = []
                    if filename.endswith('.csv'):
                        for date in dates:
                            file_path = os.path.join(path_folders, filename)

                            uf = filename.split("_")[1].split(".")[0]

                            start_date, end_date = date.split('_')
                            test_index = generate_index(start_date, end_date)
                            # test_real = get_test_real(series, start_date, end_date)
                            preds = get_preds_hybrid(file_path, date, test_index)

                            series = read_series(f'../datasets/venda/mensal/uf/{derivado}/mensal_{uf.lower()}_{derivado}.csv')
                            test_real = get_test_real(series, start_date, end_date)

                            all_preds.extend(preds.tolist())
                            all_test.extend(test_real)

                        # y_baseline = train[-horizon*1:].values
                        rmse_result = rmse(all_test, all_preds)
                        mape_result = mape(all_test, all_preds)
                        pocid_result = pocid(all_test, all_preds)
                        pbe_result = pbe(pd.Series(all_test), pd.Series(all_preds))
                        # mase_result = mase(test, preds_real, y_baseline)

                        all_pocid.append(pocid_result)
                        all_pbe.append(pbe_result)
                        all_rmse.append(rmse_result)
                        all_mape.append(mape_result)

            #realizar media de all_rmse, all_mape, all_pocid, all_pbe
            #salve em um arquivo csv com a forma de append para adicionar novas linhas
            #colunas
            #representacao, RMSE, MAPE, POCID, PBE

            #onde representacao é {model}_{transformation}
            if len(all_rmse) > 0:
                media_rmse = sum(all_rmse) / len(all_rmse)
                media_mape = sum(all_mape) / len(all_mape)
                media_pocid = sum(all_pocid) / len(all_pocid)
                media_pbe = sum(all_pbe) / len(all_pbe)

                representacao = f"{modelo}_{transformation}"

                data = {
                    "representacao": [representacao],
                    "RMSE": [media_rmse],
                    "MAPE": [media_mape],
                    "POCID": [media_pocid],
                    "PBE": [media_pbe],
                    "derivado": [derivado]
                }

                df = pd.DataFrame(data)

                os.makedirs('./metrics/', exist_ok=True)
                csv_file = f'./metrics/{derivado}.csv'

                df.to_csv(csv_file, mode='a', header=not pd.io.common.file_exists(csv_file), index=False)

ETS


arima
